In [1]:
% matplotlib inline

import pandas as pd
from datetime import datetime, timedelta
import time
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def mat_to_python_time(row):
    matlab_datenum = row['Time']
    python_datetime = datetime.fromordinal(int(matlab_datenum)) + timedelta(days=matlab_datenum%1) - timedelta(days = 366)
    return python_datetime

def time_interp_chl(uway, sfl):
    x = [time.mktime(datetime.strptime(sfl['file_time'][i],"%Y-%m-%d %H:%M:%S").timetuple()) for i in range(len(sfl))]
    x = np.asarray(x)
    xp = [time.mktime((uway['file_time'][i]).timetuple()) for i in range(len(uway))]
    xp = np.asarray(xp)
    fp = chl['Chl (ug/l)']
    fp = pd.to_numeric(fp, errors = coerce)
    
    int_chl = np.interp(x, xp, fp)
    sfl['chl'] = int_chl
    return sfl

In [3]:
# import mega dataset
filePath = '/Users/archie/Google Drive/seaflow_data/KM1_abun_sds_opp.csv'
data = pd.read_csv(filePath, sep=',')

# only keep KM1304 data
data = data[(data['cruise']=='KiloMoana_1')]

# import the underway chl data
filePath = '/Users/archie/Google Drive/seaflow_data/KM1314/KM_uway_Chl.csv'
chl = pd.read_csv(filePath, sep=',', skiprows=1)


In [4]:
# convert matlab datetime to file_time
chl['file_time'] = chl.apply(mat_to_python_time, axis=1)

In [5]:
# interpolate underway chlorophyll onto the sfl timepoints

data = time_interp_chl(chl, data)

clean = data[['cruise', 'file_time', 'chl']]
clean = clean.sort_values(by = ['cruise', 'file_time'],ascending=[1, 1])

outfile = '/Users/archie/Google Drive/seaflow_data/KM1_chl.csv'
clean.to_csv(outfile, sep = ',', index = False)